In [ ]:
# =============================================================================
# BIOCLIMATIC VARIABLES MAPPING AND VISUALIZATION
# =============================================================================

# Purpose
# - Create publication-quality maps of historical vs. future BIOCLIM variables
# - Visualize per-variable differences to highlight spatial change
# - Export figures for reporting/GIS use
#
# Requirements
# - Predictor rasters must be spatially aligned (same CRS, resolution, extent)
# - Rasters follow the naming scheme used later in this notebook
# - Variables `models`, `specie`, `region`, `savefig` are defined upstream
#
# Notes
# - Large rasters: consider tiling/chunked reads if memory is constrained
# - Colormaps are chosen to reflect variable types (temp vs precip) and differences

import os  # File system operations
import matplotlib.pyplot as plt  # Plotting and visualization
import rioxarray  # Raster I/O for xarray
import geopandas as gpd  # Geospatial data handling

In [ ]:
# Paths

figs_path = os.path.join(os.path.dirname(os.getcwd()), 'figs')
out_path = os.path.join(os.path.dirname(os.getcwd()), 'out', specie)
input_path = os.path.join(out_path, 'input')

In [ ]:
# =============================================================================
# BIOCLIMATIC VARIABLE SELECTION
# =============================================================================

# Choose which WorldClim bioclimatic variables to visualize.
# Convention: 1–11 are temperature-related, 12–19 are precipitation-related.
# Adjust `bioclim_plot` to a subset (e.g., [1, 5, 12]) to limit output.
bioclim_plot = [i for i in range(1, 20)]

# Human-readable names for figure titles and legend context.
bioclim_names = {
    1: 'Annual Mean Temperature',
    2: 'Mean Diurnal Range',
    3: 'Isothermality (×100)',
    4: 'Temperature Seasonality (standard deviation ×100)',
    5: 'Max Temperature of Warmest Month',
    6: 'Min Temperature of Coldest Month',
    7: 'Temperature Annual Range',
    8: 'Mean Temperature of Wettest Quarter',
    9: 'Mean Temperature of Driest Quarter',
    10: 'Mean Temperature of Warmest Quarter',
    11: 'Mean Temperature of Coldest Quarter',
    12: 'Annual Precipitation',
    13: 'Precipitation of Wettest Month',
    14: 'Precipitation of Driest Month',
    15: 'Precipitation Seasonality',
    16: 'Precipitation of Wettest Quarter',
    17: 'Precipitation of Driest Quarter',
    18: 'Precipitation of Warmest Quarter',
    19: 'Precipitation of Coldest Quarter'
}

# Figure layout: one row per variable; three columns for Hist, Future, Difference
nrows = len(bioclim_plot)
ncols = 3
figsize = (18, 4 * nrows)

# REQUIRE: `models` is an iterable of model prefixes that match input filenames
for model_prefix in models:
    # Shared axis and constrained layout to align panels and colorbars
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, sharex=True, sharey=True, constrained_layout=True)
    
    for idx, bc_no in enumerate(bioclim_plot):

        # Construct file paths using convention: {model}_bio_{N}_{region}[,_future].tif
        hist_file = os.path.join(input_path, f'{model_prefix}_bio_{bc_no}_{region}.tif')
        future_file = os.path.join(input_path, f'{model_prefix}_bio_{bc_no}_{region}_future.tif')
    
        # Read rasters (masked=True ensures nodata is handled for plotting/ops)
        hist_ds = rioxarray.open_rasterio(hist_file, masked=True, chunks=True)
        future_ds = rioxarray.open_rasterio(future_file, masked=True, chunks=True)
        
        # Select colormap by variable group: temperatures vs precipitation vs others
        if bc_no < 12:
            cmap = plt.cm.Spectral_r
        elif bc_no == 12:
            cmap = plt.cm.BuGn
        else:
            cmap = plt.cm.BrBG

        # Read rasters
        hist_ds = rioxarray.open_rasterio(hist_file, masked=True, chunks=True)
        future_ds = rioxarray.open_rasterio(future_file, masked=True, chunks=True)
        
        # Convert ke numpy via dask compute
        hist_data = hist_ds.isel(band=0).compute()
        future_data = future_ds.isel(band=0).compute()
        difference = future_data - hist_data
        
        # Plot
        pcol_hist = hist_data.plot.imshow(ax=ax[idx, 0], cmap=cmap, add_colorbar=False, add_labels=False)
        ax[idx, 0].set_title(f'BIO {bc_no}: {bioclim_names[bc_no]} - Historical', fontsize=12)

        pcol_future = future_data.plot.imshow(ax=ax[idx, 1], cmap=cmap, add_colorbar=False, add_labels=False)
        ax[idx, 1].set_title(f'BIO {bc_no}: {bioclim_names[bc_no]} - Future', fontsize=12)

        pcol_diff = difference.plot.imshow(ax=ax[idx, 2], cmap="coolwarm", add_colorbar=False, add_labels=False)
        ax[idx, 2].set_title(f'BIO {bc_no}: {bioclim_names[bc_no]} - Future - Hist', fontsize=12)
    
        # Colorbars: vertical for Difference, shared horizontal for Hist/Future
        cbar = fig.colorbar(pcol_diff, ax=ax[idx, 2], orientation='vertical', fraction=0.075, pad=0.01)
        cbar.ax.tick_params(labelsize=10)
        cbar = fig.colorbar(pcol_hist, ax=[ax[idx, 0], ax[idx, 1]], orientation='horizontal', fraction=0.05, pad=0.1)
        cbar.ax.tick_params(labelsize=8)

    # Save one figure per model prefix (all BIO variables stacked vertically)
    if savefig:
        file_path = os.path.join(figs_path, '07_Bioclim_%s_%s_%s.png' % (model_prefix, specie, region))   
        fig.savefig(file_path, transparent=True)